In [7]:
import re
import os
from urllib.parse import urljoin, urlparse
import requests
import pandas as pd

In [8]:
BASE_URL = "https://lom.agc.gov.my"
CSV_PATH = 'legislation_combined.csv'
PDF_FOLDER = r'C:/Users/nanadhirah/Desktop/important/legislation/legislation_pdf'
output_csv_path = 'pdf_legislation_links.csv'
os.makedirs(PDF_FOLDER, exist_ok=True)

In [9]:
def clean_url(url):
    if url:
        url = re.sub(r'/{2,}', '/', url.strip())
        if not url.startswith(('http://', 'https://')):
            url = urljoin(BASE_URL, url.lstrip('/'))
        return urljoin(BASE_URL, urlparse(url).path)
    return None

In [10]:
def download_pdf(url, save_path):
    if os.path.exists(save_path):
        print(f"PDF already exists: {save_path}")
        return False
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, "wb") as file:
                file.write(response.content)
            return True
    except Exception as e:
        print(f"Error downloading {url}: {e}")
    return False

In [5]:
#download pdf
df = pd.read_csv(CSV_PATH)
for index, row in df.iterrows():
    act_no = str(row['Act No']) if pd.notna(row['Act No']) else "UnknownActNo"
    for col in ['BM PDF Link', 'EN PDF Link', 'PDF Link']:
        if pd.notna(row.get(col)):
            url = clean_url(row[col])
            save_name = f"{index}_{act_no}_{col.split()[0]}.pdf"
            save_path = os.path.join(PDF_FOLDER, save_name)
            download_pdf(url, save_path)

FileNotFoundError: [Errno 2] No such file or directory: 'legislation_test.csv'

In [11]:
# Function to create the mapping and save as CSV
def match_pdfs_with_links(csv_path, output_csv_path, pdf_folder):
    # Step 2: Match PDFs with Links
    pdf_links = {}

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)
    
    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        act_no = str(row['Act No']) if pd.notna(row['Act No']) else "UnknownActNo"
        
        # Iterate over the PDF link columns and match
        for col in ['BM PDF Link', 'EN PDF Link', 'PDF Link']:
            if pd.notna(row.get(col)):
                # Construct the PDF file name
                pdf_file_name = f"{index}_{act_no}_{col.split()[0]}.pdf"
                
                # Check if the PDF file exists in the specified folder
                pdf_file_path = os.path.join(pdf_folder, pdf_file_name)
                if os.path.exists(pdf_file_path):
                    # If the file exists, store the link in the dictionary
                    pdf_links[pdf_file_name] = row[col]
    
    # Convert the dictionary to a DataFrame
    pdf_links_df = pd.DataFrame(pdf_links.items(), columns=['PDF File', 'Original Link'])
    
    # Save the DataFrame to a CSV file
    pdf_links_df.to_csv(output_csv_path, index=False)
    print(f"CSV saved to: {output_csv_path}")


In [12]:
match_pdfs_with_links(CSV_PATH, output_csv_path, PDF_FOLDER)

CSV saved to: pdf_legislation_links.csv
